# Wine Quality — Elastic Net Regression (one command per code cell)
# Combines L1 and L2 penalties; balances sparsity and stability.


# Imports

In [ ]:
import pandas as pd  # data handling


In [ ]:
import numpy as np  # numeric helpers


In [ ]:
from sklearn.model_selection import train_test_split, cross_validate  # split and CV


In [ ]:
from sklearn.linear_model import ElasticNet  # Elastic Net regressor


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # metrics


# Load dataset

In [ ]:
DATASET = 'red'  # 'red' or 'white'


In [ ]:
CSV_PATH = f"/Users/sb/Documents/Study-Material/Machine-Learning/winequality-{DATASET}.csv"  # path


In [ ]:
df = pd.read_csv(CSV_PATH, sep=';')  # load CSV


# Split

In [ ]:
X = df.drop(columns=['quality'])  # features


In [ ]:
y = df['quality']  # target


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # split


# Model

In [ ]:
enet = ElasticNet(alpha=0.001, l1_ratio=0.5, random_state=42, max_iter=10000)  # Elastic Net with mixed L1/L2


In [ ]:
enet.fit(X_train, y_train)  # fit model


In [ ]:
pred = enet.predict(X_test)  # predictions


# Evaluate

In [ ]:
mae = mean_absolute_error(y_test, pred)  # MAE


In [ ]:
rmse = mean_squared_error(y_test, pred, squared=False)  # RMSE


In [ ]:
r2 = r2_score(y_test, pred)  # R^2


In [ ]:
{'MAE': mae, 'RMSE': rmse, 'R2': r2}  # metrics


# Cross-validation

In [ ]:
cv = cross_validate(enet, X, y, cv=5, scoring=('r2','neg_root_mean_squared_error'))  # 5-fold CV


In [ ]:
{k: (np.mean(v), np.std(v)) for k, v in cv.items() if k.startswith('test_')}  # CV summary


# MLflow Tracking — log params, metrics, and model

In [ ]:
import mlflow  # experiment tracking


In [ ]:
import mlflow.sklearn  # sklearn flavor helpers


In [ ]:
mlflow.set_experiment('ElasticNetRegression-WineQuality')  # experiment per algorithm


In [ ]:
mlflow.start_run(run_name=f"{DATASET}")  # start run


In [ ]:
mlflow.log_params({'alpha': getattr(enet, 'alpha', None), 'l1_ratio': getattr(enet, 'l1_ratio', None), 'max_iter': getattr(enet, 'max_iter', None)})  # params


In [ ]:
mlflow.log_metric('MAE', mae)  # MAE


In [ ]:
mlflow.log_metric('RMSE', rmse)  # RMSE


In [ ]:
mlflow.log_metric('R2', r2)  # R2


In [ ]:
mlflow.sklearn.log_model(enet, artifact_path='model')  # log model


In [ ]:
mlflow.end_run()  # end run


# Persistence — save to disk and reload

In [ ]:
import os  # fs


In [ ]:
import joblib  # persistence


In [ ]:
from datetime import datetime  # timestamp


In [ ]:
os.makedirs('Machine-Learning/models/ElasticNetRegression', exist_ok=True)  # ensure dir


In [ ]:
MODEL_PATH = f"Machine-Learning/models/ElasticNetRegression/elasticnet_regression_wine_{DATASET}_" + datetime.now().strftime('%Y%m%d_%H%M%S') + ".joblib"  # path


In [ ]:
joblib.dump(enet, MODEL_PATH)  # save


In [ ]:
loaded = joblib.load(MODEL_PATH)  # load


In [ ]:
float(loaded.predict(X_test.iloc[[0]])[0])  # check


# Streamlit App — minimal predictor UI

In [ ]:
APP_CODE = """
import joblib, pandas as pd, streamlit as st
st.set_page_config(page_title='Wine — Elastic Net', page_icon='🍷')
st.title('🍷 Wine Quality — Elastic Net Regression')
MODEL_PATH = st.text_input('Model path', value=r'%s')
if MODEL_PATH and st.button('Load model'):
    try:
        st.session_state['model'] = joblib.load(MODEL_PATH)
        st.success('Model loaded')
    except Exception as e:
        st.error(f'Load failed: {e}')
if 'model' in st.session_state:
    cols = %s
    vals = {c: st.number_input(c, value=0.0) for c in cols}
    if st.button('Predict quality'):
        df = pd.DataFrame([vals])
        try:
            y = st.session_state['model'].predict(df)
            st.metric('Predicted quality', f"{float(y[0]):.3f}")
        except Exception as e:
            st.error(f'Predict failed: {e}')
""" % (MODEL_PATH, list(X.columns))


In [ ]:
open('app_streamlit_wine.py', 'w').write(APP_CODE)  # write app file


# AWS S3 Upload (commented)

In [ ]:
# import boto3


In [ ]:
# s3 = boto3.client('s3')


In [ ]:
# bucket = os.environ.get('S3_BUCKET','your-bucket')


In [ ]:
# key = 'wine-models/ElasticNetRegression/' + os.path.basename(MODEL_PATH)


In [ ]:
# s3.upload_file(MODEL_PATH, bucket, key)


In [ ]:
float(enet.predict(X_test.iloc[[0]])[0])  # single-row prediction
